In [39]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from IPython.display import clear_output

In [15]:
def load_data(data):
    arr = pd.read_csv(data, sep=' ', header=None).to_numpy()
    return arr[:,:-1], arr[:,-1]

# Questions 11-14

In [7]:
X_train, y_train = load_data('NNtrain.dat')
X_test, y_test = load_data('NNtest.dat')

In [8]:
class NeuralNet:
    def __init__(self, layers):
        self.layers = layers
    
    def del_tanh(self, x):
        return 1-np.tanh(x)*np.tanh(x)
    
    def fit(self, X_train, y_train, eta, r):
        self.weights = [2*r*np.random.random((1+d1, d2))-r 
                        for d1, d2 in zip(self.layers[:-1], self.layers[1:])]
        self.weights = [None, *self.weights]
        self.ss = [np.zeros(l) for l in self.layers]
        self.xs = [np.ones(l+1) for l in self.layers]
        self.deltas = [np.zeros(l) for l in self.layers]
        
        for t in range(50_000):
            idx = np.random.randint(0, len(X_train))
            x, y = X_train[idx], y_train[idx]
            self.forward(x)
            self.backprop(y, eta)
    
    def forward(self, x):
        self.xs[0][1:] = x
        for i in range(1, len(self.layers)):
            self.ss[i] = self.weights[i].T @ self.xs[i-1]
            self.xs[i][1:] = np.tanh(self.ss[i])
    
    def backprop(self, y, eta):
        self.deltas[-1] = -2*(y-np.tanh(self.ss[-1])) * self.del_tanh(self.ss[-1])
        self.weights[-1] = self.weights[-1] - eta * (
            self.xs[-2].reshape((-1, 1)) @ self.deltas[-1].reshape((-1, 1)).T)
        
        for i in range(1, len(self.layers)-1)[::-1]:
            self.deltas[i] = self.del_tanh(self.ss[i]) * (self.weights[i+1] @ self.deltas[i+1])[1:]
            self.weights[i] = self.weights[i] - eta * (
                self.xs[i-1].reshape((-1, 1)) @ self.deltas[i].reshape((-1, 1)).T)
    
    def predict(self, X_test, y_test):
        E_out = 0
        for i in range(len(X_test)):
            x, y = X_test[i], y_test[i]
            self.forward(x)
            y_pred = np.sign(self.xs[-1][-1])
            E_out += (y_pred != y)
        return E_out / len(X_test)

In [41]:
# Question 14
X_train, y_train = load_data('NNtrain.dat')
X_test, y_test = load_data('NNtest.dat')

d = X_train.shape[1]
eta = 0.01
r = 0.1

tot_Eout = 0
for rep in range(500):
    nn = NeuralNet([d, 8, 3, 1])
    nn.fit(X_train, y_train, eta, r)
    tot_Eout += nn.predict(X_test, y_test)
    clear_output(wait=True)
    print(f'Rep {rep}:', tot_Eout / (rep+1))
print(tot_Eout / 500)

Rep 499: 0.037807999999999904
0.037807999999999904


In [23]:
def run_exp(m, i):
    X_train, y_train = load_data('NNtrain.dat')
    X_test, y_test = load_data('NNtest.dat')
    
    d = X_train.shape[1]
    eta = 0.1
    r = 0.1
    
    avg_Eout = 0
    for rep in trange(500, position=i):
        nn = NeuralNet([d, m, 1])
        nn.fit(X_train, y_train, eta, r)
        avg_Eout += nn.predict(X_test, y_test)
    
    return (m, avg_Eout/500)

In [28]:
with ProcessPoolExecutor(max_workers=5) as exe:
    ftrs = []
    for i, m in enumerate([1, 6, 11, 16, 21]):
        exe.submit(run_exp, (m, i))
    for ftr in ftrs:
        ftr.wait()
        print(ftr.result())

ok


## Questions 15-18

In [102]:
class kNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X_train, y_train):
        self.X = X_train
        self.y = y_train
        
    def predict(self, X, y):
        y_pred = np.zeros(y.shape)
        for i in range(len(X)):
            x = X[i]
            dists = np.sum((self.X-x) ** 2, axis=1)
            assert(len(dists) == len(self.y))
            inds = np.argsort(dists)
            
            pred = 0
            for idx in inds[:self.k]:
                pred += self.y[idx]
            y_pred[i] = np.sign(pred)
        return np.sum(y != y_pred) / len(y)

In [113]:
X_train, y_train = load_data('NBORtrain.dat')
X_test, y_test = load_data('NBORtest.dat')

In [115]:
clf = kNN(1)
clf.fit(X_train, y_train)
print(clf.predict(X_train, y_train))
print(clf.predict(X_test, y_test))

0.0
0.344


In [114]:
clf = kNN(5)
clf.fit(X_train, y_train)
print(clf.predict(X_train, y_train))
print(clf.predict(X_test, y_test))

0.16
0.316


# Questions 19-20

In [139]:
X = pd.read_csv('KMeanstrain.dat', sep=' ', header=None).to_numpy()[:,:-1]

In [150]:
class kMeans:
    def __init__(self, k):
        self.k = k
    
    def fit(self, X):
        N, d = X.shape
        
        self.group = np.ones(N, dtype=int) * -1
        self.centers = []
        
        inds = np.arange(N)
        np.random.shuffle(inds)
        for i in range(self.k):
            idx = inds[i]
            self.centers.append(X[idx])
        
        no_change = False
        Ein = 0
        # it = 0
        while not no_change:
            # it += 1
            no_change = True
            Ein = 0
            
            # 1st step in optimization
            dists = [np.zeros(N) for _ in range(self.k)]
            for i in range(self.k):
                dists[i] = np.sum((X - self.centers[i]) ** 2, axis=1)
                
            # Finishing 1st step while starting 2nd step jointly
            new_centers = [np.zeros(d) for _ in range(self.k)]
            group_sizes = [0 for _ in range(self.k)]
            for i in range(N):
                chosen_group = -1
                chosen_dist = float('inf')
                for j in range(self.k):
                    if dists[j][i] < chosen_dist:
                        chosen_group = j
                        chosen_dist = dists[j][i]
                
                if self.group[i] != chosen_group:
                    no_change = False
                
                self.group[i] = chosen_group
                new_centers[chosen_group] += X[i]
                group_sizes[chosen_group] += 1
            
            # Finish 2nd step
            for i in range(self.k):
                self.centers[i] = new_centers[i] / group_sizes[i]
            
            # Compute Ein
            for i in range(N):
                center = self.centers[self.group[i]]
                Ein += np.sqrt(np.sum((X[i] - center) ** 2))
        
        # print(it)
        return Ein / N

In [155]:
avg_Ein = 0
for _ in tqdm(range(500)):
    clf = kMeans(2)
    avg_Ein += clf.fit(X)
print(avg_Ein / 500)

100%|██████████| 500/500 [00:07<00:00, 68.91it/s]

1.6112183753431983


In [157]:
avg_Ein = 0
for _ in tqdm(range(500)):
    clf = kMeans(10)
    avg_Ein += clf.fit(X)
print(avg_Ein / 500)

100%|██████████| 500/500 [00:07<00:00, 63.48it/s]

1.2594087230236506
